## langchain.agents.agent_toolkits.openapi

This is the most advanced API-calling toolset, which does include a planner so the agent can make multiple API calls if needed, and reads an OpenAPI spec.

Takes 2+ mins to run w/ gpt-4

In [19]:
from langchain.requests import Requests
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
import time


import pprint
import json
import dotenv

dotenv.load_dotenv()

pp = pprint.PrettyPrinter(indent=4)

Load the model, the JSON spec, and instantiate the agent.

In [2]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

with open("openapi.json") as f:
    raw_spec = json.load(f)
spec = reduce_openapi_spec(raw_spec)

requests_wrapper = RequestsWrapper()


planning_agent = planner.create_openapi_agent(spec, requests_wrapper, llm)

Run the query

In [3]:
%%time
user_query = "What are the phenotypes for Cystic Fibrosis?"
output = planning_agent(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the phenotypes for Cystic Fibrosis
Observation: 1. GET /search with a query param to search for Cystic Fibrosis
2. GET /disease-phenotypes with the disease ID from the search result to get a list of phenotypes associated with Cystic Fibrosis
Thought:I have a plan to get the phenotypes for Cystic Fibrosis. Now, I will execute the API calls.
Action: api_controller
Action Input: 1. GET /search with a query param to search for Cystic Fibrosis
2. GET /disease-phenotypes with the disease ID from the search result to get a list of phenotypes associated with Cystic Fibrosis

> Entering new AgentExecutor chain...
Action: requests_get
Action Input: {"url": "http://localhost:3434/search", "params": {"term": "Cystic Fibrosis"}, "output_instructions": "Extract the id and name of the first result"}
Observation: {"id": "MONDO:0009061", "name": "cystic fibrosis"}
Thought:Action: requests

In [4]:
pp.pprint(output)

{   'input': 'What are the phenotypes for Cystic Fibrosis?',
    'output': 'The phenotypes for Cystic Fibrosis are:\n'
              '1. Elevated sweat chloride\n'
              '2. Recurrent Haemophilus influenzae infections\n'
              '3. Chronic lung disease\n'
              '4. Recurrent pneumonia\n'
              '5. Airway obstruction\n'
              '6. Recurrent bronchopulmonary infections\n'
              '7. Osteopenia\n'
              '8. Osteoporosis\n'
              '9. Gastroesophageal reflux'}


## Try with gpt-3.5-turbo

I'm not sure how to get the intermediate steps for this agent similarly to the other solutions. It might not be implemented in this agent? 

Anyway, here's the same thing with gpt-3.5-turbo for comparison:

In [20]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

with open("openapi.json") as f:
    raw_spec = json.load(f)
spec = reduce_openapi_spec(raw_spec)

requests_wrapper = RequestsWrapper()

# results in error OutputParserException: Could not parse LLM output: `I have successfully executed the plan and obtained the list of phenotypes associated with Cystic Fibrosis.`
#planning_agent = planner.create_openapi_agent(spec, requests_wrapper, llm, return_intermediate_steps=True)
planning_agent = planner.create_openapi_agent(spec, requests_wrapper, llm)

#### But wait, why does this fail?

In [21]:
%%time
user_query = "What are the phenotypes for Cystic Fibrosis?"
output = planning_agent(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the phenotypes for Cystic Fibrosis
Observation: 1. GET /search with a query param to search for "Cystic Fibrosis"
2. Look for the ID of the disease in the response
3. GET /disease-phenotypes with the disease ID to get a list of phenotypes associated with Cystic Fibrosis.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /search with a query param to search for "Cystic Fibrosis"
2. Look for the ID of the disease in the response
3. GET /disease-phenotypes with the disease ID to get a list of phenotypes associated with Cystic Fibrosis.

> Entering new AgentExecutor chain...
I need to search for Cystic Fibrosis and get the disease ID, then use that ID to get a list of phenotypes associated with the disease. I should use the GET method for both API calls and extract the necessary information from the responses.

Action: requests_get
Action

OutputParserException: Could not parse LLM output: `I successfully retrieved the list of phenotypes associated with Cystic Fibrosis. I have completed the plan.`

In [22]:
pp.pprint(output)

{   'input': 'What are the phenotypes for Cystic Fibrosis?',
    'output': 'The list of 62 associations between Cystic Fibrosis and various '
              'phenotypes.'}
